In [5]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [9]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [10]:
subject = 'Espírito Santo - Consumo de Cimento (t)'
split_index = 191 #Referente aos 230 anos de input  
train_split = split_index + 1 -30

In [11]:
data = pd.read_csv('2003_mo_model_input_ES.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Espírito Santo - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Espírito Santo - IDH,Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - Produção de Cimento (t),Espírito Santo - value,Espírito Santo - Consumo de Cimento (t)
0,2003-1,8.177444,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.760059,6.342537e+07,3.523987e+06,16.799308,5.948249e+07,98.171240,0.268445,48.426
1,2003-2,8.171622,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.760222,6.350834e+07,3.527412e+06,16.807398,5.951949e+07,97.897317,0.270512,54.969
2,2003-3,8.165799,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.760385,6.359131e+07,3.530838e+06,16.815488,5.955649e+07,98.070539,0.271941,53.847
3,2003-4,8.159977,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.760548,6.367428e+07,3.534263e+06,16.823577,5.959349e+07,98.460789,0.273321,50.375
4,2003-5,8.154154,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.760710,6.375725e+07,3.537689e+06,16.831667,5.963048e+07,99.084134,0.274453,52.576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.919726,0.505935,100.170
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.756019,0.504835,85.832
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.684567,0.502971,91.318
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.664258,0.500670,77.187


In [12]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Espírito Santo - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Espírito Santo - IDH,Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - Produção de Cimento (t),Espírito Santo - value
0,-0.758586,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.007957,-1.927365,-0.943195,0.087180,-1.813287,-1.319688,-1.294991
1,-0.763437,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.978578,-1.900219,-0.892433,0.126806,-1.716680,-1.326255,-1.246951
2,-0.768288,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.949198,-1.873074,-0.841670,0.166431,-1.620072,-1.322102,-1.213715
3,-0.773139,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.919818,-1.845928,-0.790908,0.206056,-1.523465,-1.312746,-1.181649
4,-0.777990,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.890438,-1.818783,-0.740145,0.245681,-1.426858,-1.297802,-1.155335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.925704,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.304933,1.073521,-0.934891,-1.155565,-0.875236,-1.727603,1.093145
188,0.920658,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.292490,1.056445,-0.903403,-1.167749,-0.908951,-1.753915,1.108075
189,0.915611,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.280047,1.039370,-0.871915,-1.179933,-0.942665,-1.778143,1.130399
190,0.910564,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.267603,1.022295,-0.840427,-1.192116,-0.976380,-1.806324,1.153917


In [13]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      48.646
1      43.597
2      56.331
3      48.488
4      52.856
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Espírito Santo - Consumo de Cimento (t), Length: 240, dtype: float64

In [14]:
# input para treinamento
train_input = input_data.iloc[:train_split]
train_input

,Espírito Santo - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Espírito Santo - IDH,Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - Produção de Cimento (t),Espírito Santo - value
0,-0.758586,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.007957,-1.927365,-0.943195,0.087180,-1.813287,-1.319688,-1.294991
1,-0.763437,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.978578,-1.900219,-0.892433,0.126806,-1.716680,-1.326255,-1.246951
2,-0.768288,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.949198,-1.873074,-0.841670,0.166431,-1.620072,-1.322102,-1.213715
3,-0.773139,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.919818,-1.845928,-0.790908,0.206056,-1.523465,-1.312746,-1.181649
4,-0.777990,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.890438,-1.818783,-0.740145,0.245681,-1.426858,-1.297802,-1.155335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.538868,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.408727,1.007109,-1.505857,-1.639139,-1.626063,-0.138284,1.037783
158,1.502037,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.408716,1.017654,-1.496536,-1.616758,-1.583348,-0.199466,1.039895
159,1.465205,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.408704,1.028199,-1.487214,-1.594377,-1.540633,-0.263743,1.042387
160,1.428374,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.408692,1.038744,-1.477892,-1.571996,-1.497917,-0.329315,1.044800


In [15]:
# Alvo para treinamento
train_target = target_data.iloc[:train_split]
train_target

0       48.646
1       43.597
2       56.331
3       48.488
4       52.856
        ...   
157     77.958
158    105.434
159     86.699
160     90.574
161     85.148
Name: Espírito Santo - Consumo de Cimento (t), Length: 162, dtype: float64

In [16]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,Espírito Santo - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Espírito Santo - IDH,Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - Produção de Cimento (t),Espírito Santo - value
162,1.354711,-0.601510,-0.653307,0.157879,-1.255326,1.088045,0.763466,-1.213929,1.408668,1.059833,-1.459249,-1.527235,-1.412487,-0.443908,1.054208
163,1.317880,-0.786068,-0.653814,0.548855,-1.118679,1.099377,0.752299,-1.292173,1.408656,1.070378,-1.449927,-1.504854,-1.369772,-0.502480,1.058892
164,1.281048,-0.830387,-0.645656,0.385685,-0.936336,1.110709,0.741131,-1.324219,1.408644,1.080923,-1.440605,-1.482474,-1.327057,-0.562163,1.064897
165,1.244217,-0.801089,-0.634274,0.420965,-0.931790,1.122042,0.729964,-1.344446,1.408633,1.091467,-1.431284,-1.460093,-1.284342,-0.621189,1.070528
166,1.207386,-0.959917,-0.624035,0.548870,-1.168522,1.133374,0.718796,-1.381638,1.408621,1.102012,-1.421962,-1.437712,-1.241626,-0.678626,1.078723
167,1.170554,-1.022309,-0.619791,0.222499,-1.285611,1.144706,0.707629,-1.411208,1.408609,1.112557,-1.412640,-1.415332,-1.198911,-0.738949,1.083845
168,1.133723,-1.074401,-0.620112,-0.614754,-1.446316,1.156038,0.696461,-1.412953,1.408597,1.123102,-1.403319,-1.392951,-1.156196,-0.795709,1.085740
169,1.119332,-1.119597,-0.615973,-0.478229,-1.357832,1.167287,0.681823,-1.491464,1.407217,1.128930,-1.382651,-1.366062,-1.113116,-0.855369,1.087571
170,1.104941,-1.078648,-0.609777,-0.393157,-1.235682,1.178536,0.667184,-1.573805,1.405837,1.134759,-1.361984,-1.339172,-1.070035,-0.908834,1.090764
171,1.090550,-1.055426,-0.602141,-0.557552,-1.065340,1.189784,0.652545,-1.564950,1.404457,1.140588,-1.341316,-1.312283,-1.026955,-0.975958,1.092987


In [17]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

162     86.678
163    100.174
164     89.777
165     94.810
166     93.446
167     87.540
168     96.706
169     74.086
170     92.676
171     82.865
172     71.180
173    108.826
174    105.694
175    107.106
176    101.893
177    110.551
178    100.587
179     94.220
180    111.377
181     98.614
182     94.022
183     91.546
184     95.674
185     88.927
186    105.688
187    104.711
188     97.530
189     99.651
190     76.710
191     75.456
Name: Espírito Santo - Consumo de Cimento (t), dtype: float64

In [22]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(split_factor):
        pos = df.shape[0] - (i*div_factor + 1)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [25]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    train, train_val = validation_splitter(train_input, 6)
    target,target_val = validation_splitter(train_target, 6)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val, target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [26]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [27]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[458770814, 729012944, 999423022, 851237050, 3449099398, 3814093924, 1067991563, 350843525, 3579841117, 1114291685, 978655544, 115057275, 3515421820, 99858558, 3225141884, 882343996, 2974221653, 2227943565, 3271633166, 1580399542, 4249218657, 904858025, 2020624673, 761295842, 4087403555]


Step: 0 ___________________________________________
val_loss: 43.467620849609375
winner_seed: 458770814


Step: 1 ___________________________________________
val_loss: 52.40235900878906


Step: 2 ___________________________________________
val_loss: 47.42789840698242


Step: 3 ___________________________________________
val_loss: 52.19179916381836


Step: 4 ___________________________________________
val_loss: 46.98338317871094


Step: 5 ___________________________________________
val_loss: 54.72330093383789


Step: 6 ___________________________________________
val_loss: 57.99717712402344


Step: 7 ___________________________________________
val_loss: 42.332298278808594
winner_seed: 350843525


Step:

In [28]:
trained_model, history = neural_network_model(train_input, 
                                              train_target, 
                                              want_verbose=1, 
                                              seed=winner_seed)

Epoch 1/10000
5/5 [==============================] - 1s 140ms/step - loss: 9449.3672 - val_loss: 7217.7256
Epoch 2/10000
5/5 [==============================] - 0s 9ms/step - loss: 7310.7397 - val_loss: 7655.9839
Epoch 3/10000
5/5 [==============================] - 0s 8ms/step - loss: 7585.8975 - val_loss: 7279.8403
Epoch 4/10000
5/5 [==============================] - 0s 7ms/step - loss: 6032.3896 - val_loss: 5193.8223
Epoch 5/10000
5/5 [==============================] - 0s 8ms/step - loss: 5677.0938 - val_loss: 5651.0386
Epoch 6/10000
5/5 [==============================] - 0s 8ms/step - loss: 5143.7715 - val_loss: 8164.1572
Epoch 7/10000
5/5 [==============================] - 0s 8ms/step - loss: 5142.5083 - val_loss: 6193.3354
Epoch 8/10000
5/5 [==============================] - 0s 8ms/step - loss: 4168.8335 - val_loss: 4006.7637
Epoch 9/10000
5/5 [==============================] - 0s 8ms/step - loss: 3706.7236 - val_loss: 2994.2375
Epoch 10/10000
5/5 [==============================] -

5/5 [==============================] - 0s 8ms/step - loss: 61.1251 - val_loss: 75.3874
Epoch 81/10000
5/5 [==============================] - 0s 8ms/step - loss: 62.3818 - val_loss: 52.8894
Epoch 82/10000
5/5 [==============================] - 0s 8ms/step - loss: 75.3727 - val_loss: 107.4694
Epoch 83/10000
5/5 [==============================] - 0s 8ms/step - loss: 65.4629 - val_loss: 117.9773
Epoch 84/10000
5/5 [==============================] - 0s 8ms/step - loss: 62.6181 - val_loss: 64.9490
Epoch 85/10000
5/5 [==============================] - 0s 9ms/step - loss: 68.7900 - val_loss: 79.1821
Epoch 86/10000
5/5 [==============================] - 0s 9ms/step - loss: 66.2287 - val_loss: 69.2237
Epoch 87/10000
5/5 [==============================] - 0s 9ms/step - loss: 68.3109 - val_loss: 66.8998
Epoch 88/10000
5/5 [==============================] - 0s 8ms/step - loss: 63.7754 - val_loss: 74.9334
Epoch 89/10000
5/5 [==============================] - 0s 8ms/step - loss: 72.3340 - val_loss: 7

5/5 [==============================] - 0s 7ms/step - loss: 65.5211 - val_loss: 50.0341
Epoch 160/10000
5/5 [==============================] - 0s 8ms/step - loss: 63.6770 - val_loss: 47.8593
Epoch 161/10000
5/5 [==============================] - 0s 7ms/step - loss: 60.0620 - val_loss: 91.6595
Epoch 162/10000
5/5 [==============================] - 0s 8ms/step - loss: 59.6299 - val_loss: 96.1176
Epoch 163/10000
5/5 [==============================] - 0s 8ms/step - loss: 58.2878 - val_loss: 59.3497
Epoch 164/10000
5/5 [==============================] - 0s 8ms/step - loss: 57.3605 - val_loss: 48.1470
Epoch 165/10000
5/5 [==============================] - 0s 7ms/step - loss: 55.3171 - val_loss: 62.2130
Epoch 166/10000
5/5 [==============================] - 0s 7ms/step - loss: 56.8389 - val_loss: 50.7164
Epoch 167/10000
5/5 [==============================] - 0s 8ms/step - loss: 54.6445 - val_loss: 61.3358
Epoch 168/10000
5/5 [==============================] - 0s 8ms/step - loss: 51.8224 - val_

2023-10-14 17:05:54.391130: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


5/5 [==============================] - 0s 8ms/step - loss: 59.3928 - val_loss: 81.4210
Epoch 187/10000
5/5 [==============================] - 0s 9ms/step - loss: 66.4918 - val_loss: 77.0186
Epoch 188/10000
5/5 [==============================] - 0s 9ms/step - loss: 56.0296 - val_loss: 44.2227
Epoch 189/10000
5/5 [==============================] - 0s 8ms/step - loss: 56.4759 - val_loss: 56.7924
Epoch 190/10000
5/5 [==============================] - 0s 7ms/step - loss: 62.6713 - val_loss: 53.3478
Epoch 191/10000
5/5 [==============================] - 0s 7ms/step - loss: 74.3766 - val_loss: 55.3765
Epoch 192/10000
5/5 [==============================] - 0s 8ms/step - loss: 74.8358 - val_loss: 48.4307
Epoch 193/10000
5/5 [==============================] - 0s 7ms/step - loss: 62.4315 - val_loss: 82.8220
Epoch 194/10000
5/5 [==============================] - 0s 8ms/step - loss: 61.9510 - val_loss: 83.2380
Epoch 195/10000
5/5 [==============================] - 0s 9ms/step - loss: 63.9579 - val_

...

5/5 [==============================] - 0s 7ms/step - loss: 44.0598 - val_loss: 77.7914
Epoch 1841/10000
5/5 [==============================] - 0s 7ms/step - loss: 41.5264 - val_loss: 52.7961
Epoch 1842/10000
5/5 [==============================] - 0s 7ms/step - loss: 41.4145 - val_loss: 65.8939
Epoch 1843/10000
5/5 [==============================] - 0s 6ms/step - loss: 40.5204 - val_loss: 41.3447
Epoch 1844/10000
5/5 [==============================] - 0s 6ms/step - loss: 45.4039 - val_loss: 66.1984
Epoch 1845/10000
5/5 [==============================] - 0s 5ms/step - loss: 40.3939 - val_loss: 56.6719
Epoch 1846/10000
5/5 [==============================] - 0s 5ms/step - loss: 42.5447 - val_loss: 58.8498
Epoch 1847/10000
5/5 [==============================] - 0s 6ms/step - loss: 41.5345 - val_loss: 77.0200
Epoch 1848/10000
5/5 [==============================] - 0s 6ms/step - loss: 46.5924 - val_loss: 77.4863
Epoch 1849/10000
5/5 [==============================] - 0s 6ms/step - loss: 42.87

Epoch 1919/10000
5/5 [==============================] - 0s 6ms/step - loss: 44.1139 - val_loss: 61.2376
Epoch 1920/10000
5/5 [==============================] - 0s 7ms/step - loss: 41.0733 - val_loss: 80.8451
Epoch 1921/10000
5/5 [==============================] - 0s 6ms/step - loss: 43.6113 - val_loss: 49.9140
Epoch 1922/10000
5/5 [==============================] - 0s 11ms/step - loss: 45.8152 - val_loss: 65.3459
Epoch 1923/10000
5/5 [==============================] - 0s 6ms/step - loss: 44.8624 - val_loss: 60.6484
Epoch 1924/10000
5/5 [==============================] - 0s 6ms/step - loss: 41.3819 - val_loss: 77.5714
Epoch 1925/10000
5/5 [==============================] - 0s 6ms/step - loss: 43.2159 - val_loss: 54.9880
Epoch 1926/10000
5/5 [==============================] - 0s 6ms/step - loss: 42.2791 - val_loss: 73.6381
Epoch 1927/10000
5/5 [==============================] - 0s 6ms/step - loss: 44.0073 - val_loss: 45.8659
Epoch 1928/10000
5/5 [==============================] - 0s 6ms/

5/5 [==============================] - 0s 12ms/step - loss: 48.9721 - val_loss: 88.9824
Epoch 1998/10000
5/5 [==============================] - 0s 8ms/step - loss: 44.3548 - val_loss: 49.9498
Epoch 1999/10000
5/5 [==============================] - 0s 9ms/step - loss: 40.4226 - val_loss: 44.5750
Epoch 2000/10000
5/5 [==============================] - 0s 9ms/step - loss: 47.3315 - val_loss: 82.1114
Epoch 2001/10000
5/5 [==============================] - 0s 8ms/step - loss: 46.5752 - val_loss: 72.0775
Epoch 2002/10000
5/5 [==============================] - 0s 9ms/step - loss: 43.3483 - val_loss: 63.1329
Epoch 2003/10000
5/5 [==============================] - 0s 8ms/step - loss: 42.3936 - val_loss: 77.1619
Epoch 2004/10000
5/5 [==============================] - 0s 8ms/step - loss: 51.1483 - val_loss: 57.5027
Epoch 2005/10000
5/5 [==============================] - 0s 12ms/step - loss: 43.8614 - val_loss: 101.5524
Epoch 2006/10000
5/5 [==============================] - 0s 9ms/step - loss: 49

Epoch 2076/10000
5/5 [==============================] - 0s 7ms/step - loss: 41.8876 - val_loss: 49.8563
Epoch 2077/10000
5/5 [==============================] - 0s 8ms/step - loss: 42.3559 - val_loss: 48.9206
Epoch 2078/10000
5/5 [==============================] - 0s 8ms/step - loss: 46.0701 - val_loss: 54.0119
Epoch 2079/10000
5/5 [==============================] - 0s 8ms/step - loss: 43.5309 - val_loss: 71.9934
Epoch 2080/10000
5/5 [==============================] - 0s 8ms/step - loss: 42.5441 - val_loss: 84.8175
Epoch 2081/10000
5/5 [==============================] - 0s 7ms/step - loss: 41.6661 - val_loss: 49.3765
Epoch 2082/10000
5/5 [==============================] - 0s 7ms/step - loss: 42.7744 - val_loss: 58.5759
Epoch 2083/10000
5/5 [==============================] - 0s 8ms/step - loss: 45.6746 - val_loss: 56.7662
Epoch 2084/10000
5/5 [==============================] - 0s 10ms/step - loss: 43.1398 - val_loss: 50.9478
Epoch 2085/10000
5/5 [==============================] - 0s 9ms/

5/5 [==============================] - 0s 9ms/step - loss: 50.4445 - val_loss: 116.2805
Epoch 2155/10000
5/5 [==============================] - 0s 7ms/step - loss: 46.3436 - val_loss: 86.4649
Epoch 2156/10000
5/5 [==============================] - 0s 9ms/step - loss: 41.2959 - val_loss: 77.3417
Epoch 2157/10000
5/5 [==============================] - 0s 8ms/step - loss: 42.6302 - val_loss: 83.1445
Epoch 2158/10000
5/5 [==============================] - 0s 8ms/step - loss: 42.0556 - val_loss: 58.8301
Epoch 2159/10000
5/5 [==============================] - 0s 9ms/step - loss: 51.2275 - val_loss: 53.7812
Epoch 2160/10000
5/5 [==============================] - 0s 12ms/step - loss: 49.3864 - val_loss: 97.2636
Epoch 2160: early stopping


In [29]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [30]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            test_input, 
                                            test_target, 
                                            train_split)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 19ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,91.670593,91.670525,91.670448,91.67041,91.669357,89.812881,1.741394,2.583832,2.544014,3.252556,1.48637,61.985703,0.924728,-1.39756,-1.827866,-1.964546,0.121807,-3.750622,-5.058132,-4.52375,-5.064434,-5.338707,-6.595284,-9.79287,-12.288124,-12.208084,-12.224022,-12.674141,-12.338783,-12.106552
Target,86.678,100.174,89.777,94.81,93.446,87.54,96.706,74.086,92.676,82.865,71.18,108.826,105.694,107.106,101.893,110.551,100.587,94.22,111.377,98.614,94.022,91.546,95.674,88.927,105.688,104.711,97.53,99.651,76.71,75.456
Error,4.992592,8.503479,1.893448,3.139587,1.776642,2.272881,94.964607,71.502167,90.131989,79.612442,69.693634,46.840294,104.769272,108.503563,103.720863,112.515549,100.465195,97.970627,116.435135,103.137749,99.086441,96.884705,102.269287,98.719872,117.976128,116.919083,109.754021,112.325142,89.048782,87.562553


In [31]:
display(mae)
display(mape)

78.44626

0.82664746

In [32]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [33]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             train_split)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 20ms/step
Ano-0: |Prediction[[621.75806]] - Target[1078.7640000000001]| =  Error: [[457.00598]]; MAPE:[[0.4236385]]
1/1 [==============================] - 0s 16ms/step
Ano-0: |Prediction[[-44.267235]] - Target[1200.211]| =  Error: [[1244.4783]]; MAPE:[[1.0368829]]
1/1 [==============================] - 0s 18ms/step
Ano-5: |Prediction[[-73.83971]] - Target[559.746]| =  Error: [[633.5857]]; MAPE:[[1.1319165]]


[array([[457.00598]], dtype=float32),
 array([[1244.4783]], dtype=float32),
 array([[633.5857]], dtype=float32)]

778.3566

0.86414593